In [ ]:
!pip install peft torch
!pip install mistralai together
!pip install openai==0.28
!pip install friendli-client
!pip install transformers==4.45.1
!pip install tiktoken
!pip install transformers_stream_generator

In [ ]:
# Import necessary libraries
import csv
import json
import os
import pandas as pd
import re
import torch
import logging
import openai
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from mistralai import Mistral
from together import Together
from huggingface_hub import login
from friendli import Friendli


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Suppress all warnings from transformers
logging.getLogger("transformers").setLevel(logging.CRITICAL)

In [ ]:
# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU device {device}.")
else:
    print("CUDA is not available. Using CPU.")

In [ ]:
# Load API keys from config
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

In [ ]:
OPENAI_API_KEY = config.get('openai_api_key', None)
MISTRAL_API_KEY = config.get('mistral_api_key', None)
TOGETHER_API_KEY = config.get('together_ai_api_key', None)
HF_API_KEY = config.get('huggingface_api_key', None)
FRIENDLI_API_TOKEN = config.get('friendli_api_token', None)

In [ ]:
login(token=HF_API_KEY)

In [ ]:
# Ollama REST API endpoint (if using Ollama)
OLLAMA_API_URL = 'http://localhost:11434/api/generate'

# Initialize the Mistral client
mistral_client = Mistral(api_key=MISTRAL_API_KEY)

# Initialize the Together client
together_client = Together(api_key=TOGETHER_API_KEY)

# Initialize the OpenAI GPT-4o client is handled via the transformers library

# Define the model configurations
MODEL_CONFIGS = [
    # {
    #     "name": "GPT-4o",
    #     "type": "openai",
    #     "model_name": "gpt-4o",  # OpenAI's GPT-4o identifier
    #     "api": "openai"
    # },
    # {
    #     "name": "Mistral_Large_2",
    #     "type": "mistral",
    #     "model_name": "mistral-large-latest",
    #     "api": "mistral"
    # },
    # {
    #     "name": "LLaMA_3.1-8B",
    #     "type": "llama",
    #     "model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    #     "api": "together"
    # },
]

# Function to initialize models
def initialize_models(model_configs, device):
    """Initialize models based on the provided configurations."""
    initialized_models = []
    for config in model_configs:
        try:
            api_type = config.get("api", "unknown")
            
            if api_type == "openai":
                print(f"Setting up OpenAI for {config['name']}...")
                initialized_models.append(config)
            elif api_type == "mistral":
                print(f"Setting up Mistral for {config['name']}...")
                initialized_models.append(config)
            elif api_type == "together":
                print(f"Setting up Together AI for {config['name']}...")
                initialized_models.append(config)
            elif api_type == "huggingface":
                print(f"Setting up HuggingFace model for {config['name']}...")
                initialized_models.append(config)
            elif api_type == "friendli":
                endpoint_type = config.get("endpoint_type", "serverless")
                if endpoint_type == "serverless":
                    print(f"Setting up Friendli serverless model for {config['name']}...")
                elif endpoint_type == "dedicated":
                    print(f"Setting up Friendli dedicated endpoint for {config['name']}...")
                elif endpoint_type == "container":
                    print(f"Setting up Friendli container for {config['name']}...")
                initialized_models.append(config)
            else:
                print(f"Unknown API type for model {config['name']}. Skipping.")
        except Exception as e:
            print(f"Error setting up model {config['name']}: {e}")
    return initialized_models

In [ ]:
# Define the inference engine for OpenAI GPT-4o
class OpenAIInferenceEngine:
    def __init__(self, model_name="gpt-4o"):
        self.model_name = model_name

    def generate_response(self, system_prompt, user_prompt):
        """
        Generate a response using OpenAI's GPT-4o.

        Args:
            system_prompt (str): System-level instructions.
            user_prompt (str): User's input or question.

        Returns:
            str: Generated response from GPT-4o.
        """
        try:
            full_prompt = f"{system_prompt}\n\n{user_prompt}"
            response = openai.ChatCompletion.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
                max_tokens=1024,
                top_p=1.0,
                frequency_penalty=0,
                presence_penalty=0
            )
            answer = response['choices'][0]['message']['content'].strip()
            return answer
        except Exception as e:
            print(f"Error generating response with {self.model_name}: {e}")
            return "Error generating response."

# Define the inference engine for Mistral Large 2
class MistralInferenceEngine:
    def __init__(self, client, model_name="mistral-large-latest"):
        self.client = client
        self.model_name = model_name

    def generate_response(self, system_prompt, user_prompt):
        """
        Generate a response using Mistral Large 2.

        Args:
            system_prompt (str): System-level instructions.
            user_prompt (str): User's input or question.

        Returns:
            str: Generated response from Mistral Large 2.
        """
        try:
            full_prompt = f"{system_prompt}\n\n{user_prompt}"
            response = self.client.chat.complete(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
                max_tokens=1024,
                top_p=1.0,
                frequency_penalty=0,
                presence_penalty=0
            )
            answer = response.choices[0].message.content.strip()
            return answer
        except Exception as e:
            print(f"Error generating response with {self.model_name}: {e}")
            return "Error generating response."

# Define the inference engine for LLaMA 3.1-8B via Together AI
class LLaMAInferenceEngine:
    def __init__(self, client, model_name="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"):
        self.client = client
        self.model_name = model_name

    def generate_response(self, system_prompt, user_prompt):
        """
        Generate a response using LLaMA 3.1-8B via Together AI.

        Args:
            system_prompt (str): System-level instructions.
            user_prompt (str): User's input or question.

        Returns:
            str: Generated response from LLaMA 3.1-8B.
        """
        try:
            full_prompt = f"{system_prompt}\n\n{user_prompt}"
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
                max_tokens=1024,
                top_p=1.0,
                frequency_penalty=0,
                presence_penalty=0
            )
            answer = response.choices[0].message.content.strip()
            return answer
        except Exception as e:
            print(f"Error generating response with {self.model_name}: {e}")
            return "Error generating response."
        
# HuggingFace direct model inference engine
class HuggingFaceInferenceEngine:
   def __init__(self, model_path, device="cuda"):
       """
       Initialize a Hugging Face model directly from the provided path.
      
       Args:
           model_path (str): The path or identifier of the model on HuggingFace Hub.
           device (str): The device to load the model on ("cuda" or "cpu").
       """
       self.model_path = model_path
       self.device = device
       print(f"Loading HuggingFace model from {model_path}...")
      
       # Two options for loading - pipeline (easier) or direct model loading (more customizable)
       # Option 1: Using pipeline (simpler)
       self.pipe = pipeline(
           "text-generation",
           model=model_path,
           device_map="auto" if device == "cuda" else "cpu",
           max_new_tokens=1024
       )
      
       # Option 2: Direct model and tokenizer loading (more control)
       self.tokenizer = AutoTokenizer.from_pretrained(model_path)
       self.model = AutoModelForCausalLM.from_pretrained(
           model_path,
           device_map="auto" if device == "cuda" else "cpu",
           torch_dtype="auto"
       )
       print(f"Successfully loaded model from {model_path}")

   def generate_response(self, system_prompt, user_prompt):
       """
       Generate a response using the HuggingFace model.
      
       Args:
           system_prompt (str): System-level instructions.
           user_prompt (str): User's input or question.
          
       Returns:
           str: Generated response from the model.
       """
       try:
           # Prepare messages in the format expected by the model
           messages = [
               {"role": "system", "content": system_prompt},
               {"role": "user", "content": user_prompt}
           ]
          
           # Generate using the pipeline
           result = self.pipe(messages, do_sample=True, temperature=0, top_p=1.0)
          
           # Extract the generated text from the result
           if isinstance(result, list) and len(result) > 0:
               if "generated_text" in result[0]:
                   # Standard output format
                   response = result[0]["generated_text"]
               else:
                   # Extract the last message content if it's a conversation
                   response = result[0]
           else:
               response = str(result)
              
           # Clean up the response to get just the assistant's reply
           # This may need adjustment based on the exact model's output format
           if isinstance(response, str) and "assistant" in response.lower():
               # Try to extract just the assistant's part
               parts = response.split("assistant")
               if len(parts) > 1:
                   response = parts[1].strip()
                   if response.startswith(":"):
                       response = response[1:].strip()
          
           return response.strip()
      
       except Exception as e:
           print(f"Error generating response with HuggingFace model: {e}")
           return f"Error generating response: {str(e)}"
       
# FriendliInferenceEngine class for handling all Friendli deployment types
class FriendliInferenceEngine:
    def __init__(self, token, model_name=None, endpoint_id=None, team_id=None, 
                 adapter_route=None, base_url=None, use_container=False):
        """
        Initialize the appropriate Friendli client based on deployment type.
        
        Args:
            token: Friendli API token (not needed for container mode)
            model_name: Name for serverless models (e.g., meta-llama-3.1-8b-instruct)
            endpoint_id: ID for dedicated endpoint custom models
            team_id: Team ID for dedicated endpoints
            adapter_route: Adapter route for Multi-LoRA models
            base_url: URL for container deployment
            use_container: Flag for container mode
        """
        
        # Store parameters
        self.token = token
        self.model_name = model_name
        self.endpoint_id = endpoint_id
        self.adapter_route = adapter_route
        self.use_container = use_container
        
        # Container mode (self-hosted)
        if use_container:
            self.client = Friendli(base_url=base_url or "http://0.0.0.0:8000")
            self.model_identifier = adapter_route if adapter_route else None
            print(f"Using Friendli Container at {base_url or 'http://0.0.0.0:8000'}")
            
        # Dedicated endpoint mode (custom models)
        elif endpoint_id:
            self.client = Friendli(token=token, team_id=team_id, use_dedicated_endpoint=True)
            if adapter_route:
                # Multi-LoRA model
                self.model_identifier = f"{endpoint_id}:{adapter_route}"
            else:
                # Regular dedicated endpoint
                self.model_identifier = endpoint_id
            print(f"Using Friendli Dedicated Endpoint: {self.model_identifier}")
            
        # Serverless mode (public models like Llama)
        else:
            self.client = Friendli(token=token)
            self.model_identifier = model_name
            print(f"Using Friendli Serverless model: {model_name}")
    
    def generate_response(self, system_prompt, user_prompt):
        """Generate a response from the Friendli model."""
        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            
            # Handle different API endpoints based on deployment type
            if self.adapter_route and not self.use_container:
                # Multi-LoRA models on dedicated endpoints use the lora API
                response = self.client.lora.completions.create(
                    model=self.model_identifier,
                    messages=messages,
                    temperature=0,
                    max_tokens=1024
                )
            else:
                # Standard chat completions API for all other cases
                response = self.client.chat.completions.create(
                    model=self.model_identifier if self.model_identifier else None,
                    messages=messages,
                    temperature=0,
                    max_tokens=1024
                )
            
            answer = response.choices[0].message.content.strip()
            return answer
            
        except Exception as e:
            print(f"Error with Friendli model: {e}")
            return f"Error: {str(e)}"
    
    def close(self):
        """Close the client to release resources."""
        if hasattr(self, 'client'):
            self.client.close()
# Define the InferenceEngine wrapper
class InferenceEngine:
    def __init__(self, model_info):
        """Initialize the appropriate inference engine based on model configuration."""
        self.name = model_info["name"]
        self.type = model_info["type"]
        self.api = model_info.get("api", None)
        
        if self.api == "openai":
            self.model_name = model_info["model_name"]
            self.engine = OpenAIInferenceEngine(model_name=self.model_name)
        elif self.api == "mistral":
            self.model_name = model_info["model_name"]
            self.engine = MistralInferenceEngine(client=mistral_client, model_name=self.model_name)
        elif self.api == "together":
            self.model_name = model_info["model_name"]
            self.engine = LLaMAInferenceEngine(client=together_client, model_name=self.model_name)
        elif self.api == "huggingface":
            self.model_name = model_info["model_name"]
            self.engine = HuggingFaceInferenceEngine(model_path=self.model_name, device=device)
        elif self.api == "friendli":
            # Handle different Friendli deployment types
            endpoint_type = model_info.get("endpoint_type", "serverless")
            
            if endpoint_type == "serverless":
                # Public models on serverless infrastructure
                self.model_name = model_info["model_name"]
                self.engine = FriendliInferenceEngine(
                    token=FRIENDLI_API_TOKEN,
                    model_name=self.model_name
                )
            elif endpoint_type == "dedicated":
                # Custom models on dedicated endpoints
                self.model_name = model_info.get("endpoint_id", "")
                self.engine = FriendliInferenceEngine(
                    token=FRIENDLI_API_TOKEN,
                    endpoint_id=model_info.get("endpoint_id"),
                    team_id=model_info.get("team_id"),
                    adapter_route=model_info.get("adapter_route")
                )
            elif endpoint_type == "container":
                # Self-hosted container deployments
                self.model_name = "container"
                self.engine = FriendliInferenceEngine(
                    token=None,  # Not needed for container
                    base_url=model_info.get("base_url"),
                    adapter_route=model_info.get("adapter_name"),
                    use_container=True
                )
        else:
            raise ValueError(f"Unsupported API type: {self.api}")

    def generate_response(self, system_prompt, user_prompt):
        """Generate a response using the underlying inference engine."""
        return self.engine.generate_response(system_prompt, user_prompt)

# Add a new HuggingFace model to the MODEL_CONFIGS list
def add_huggingface_model(model_name, display_name=None):
   """
   Add a HuggingFace model to the MODEL_CONFIGS list.
  
   Args:
       model_name (str): The model path/name on HuggingFace Hub.
       display_name (str, optional): A custom display name for the model.
      
   Returns:
       dict: The model configuration dictionary.
   """
   display_name = display_name or f"HF_{model_name.split('/')[-1]}"
   model_config = {
       "name": display_name,
       "type": "huggingface",
       "model_name": model_name,
       "api": "huggingface"
   }
   MODEL_CONFIGS.append(model_config)
   print(f"Added HuggingFace model {model_name} to MODEL_CONFIGS")
   return model_config

# Functions to add different types of Friendli models
def add_friendli_serverless(model_name, display_name=None):
    """
    Add a Friendli serverless endpoint model to the MODEL_CONFIGS list.
    
    Args:
        model_name (str): The model name (e.g., "meta-llama-3.1-8b-instruct")
        display_name (str, optional): A custom display name for the model.
        
    Returns:
        dict: The model configuration dictionary.
    """
    display_name = display_name or f"Friendli_{model_name.replace('-', '_')}"
    model_config = {
        "name": display_name,
        "type": "friendli_serverless",
        "model_name": model_name,
        "api": "friendli",
        "endpoint_type": "serverless"
    }
    MODEL_CONFIGS.append(model_config)
    print(f"Added Friendli serverless model {model_name} to MODEL_CONFIGS")
    return model_config

def add_friendli_dedicated(endpoint_id, display_name=None, team_id=None, adapter_route=None):
    """
    Add a Friendli dedicated endpoint model to the MODEL_CONFIGS list.
    
    Args:
        endpoint_id (str): The endpoint ID for the dedicated model
        display_name (str, optional): A custom display name for the model.
        team_id (str, optional): The team ID for the dedicated endpoint
        adapter_route (str, optional): Adapter route for Multi-LoRA models
        
    Returns:
        dict: The model configuration dictionary.
    """
    display_name = display_name or f"Friendli_Custom_{endpoint_id}"
    model_config = {
        "name": display_name,
        "type": "friendli_dedicated",
        "endpoint_id": endpoint_id,
        "api": "friendli",
        "endpoint_type": "dedicated",
        "team_id": team_id,
        "adapter_route": adapter_route
    }
    MODEL_CONFIGS.append(model_config)
    print(f"Added Friendli dedicated endpoint {endpoint_id} to MODEL_CONFIGS")
    return model_config

def add_friendli_container(base_url, display_name=None, adapter_name=None):
    """
    Add a Friendli container deployment model to the MODEL_CONFIGS list.
    
    Args:
        base_url (str): The base URL for the container (e.g., "http://0.0.0.0:8000")
        display_name (str, optional): A custom display name for the model.
        adapter_name (str, optional): The adapter name for Multi-LoRA models
        
    Returns:
        dict: The model configuration dictionary.
    """
    display_name = display_name or f"Friendli_Container"
    model_config = {
        "name": display_name,
        "type": "friendli_container",
        "base_url": base_url,
        "api": "friendli",
        "endpoint_type": "container",
        "adapter_name": adapter_name
    }
    MODEL_CONFIGS.append(model_config)
    print(f"Added Friendli container at {base_url} to MODEL_CONFIGS")
    return model_config

In [ ]:
add_huggingface_model("meta-llama/Llama-3.2-3B-Instruct") # change the path for your model

In [ ]:
# 1. public model (like Llama) on Friendli's serverless infrastructure
add_friendli_serverless("meta-llama-3.1-8b-instruct")

# 2. custom model on a dedicated endpoint
add_friendli_dedicated(
    endpoint_id="your-endpoint-id",  # Replace with your actual endpoint ID
    team_id="your-team-id"           # Replace with your team ID if needed
)

# 3. custom model with LoRA adapter on a dedicated endpoint
add_friendli_dedicated(
    endpoint_id="your-endpoint-id",
    adapter_route="your-adapter-route"
)

# 4. self-hosted Friendli Container deployment
add_friendli_container("http://your-container-url:8000")

In [ ]:
# Initialize all models
initialized_models_info = initialize_models(MODEL_CONFIGS, device)
inference_engines = [InferenceEngine(model_info) for model_info in initialized_models_info]

In [ ]:
# Define the base Task class
class Task:
    def __init__(self, inference_engine, file_path, col_question, col_label, csv_output_path, json_output_path):
        """
        Initializes a task with the given parameters.

        Args:
            inference_engine (InferenceEngine): The inference engine to generate responses.
            file_path (str): Path to the CSV file with question data.
            col_question (str): Name of the column containing the questions.
            col_label (str): Name of the column containing the labels/answers.
            csv_output_path (str): Path to save CSV output.
            json_output_path (str): Path to save JSON output.
        """
        self.inference = inference_engine
        self.file_path = file_path
        self.col_question = col_question
        self.col_label = col_label
        self.csv_output_path = csv_output_path
        self.json_output_path = json_output_path

    def save_results(self, answers, results):
        """
        Saves task results to CSV and JSON files.

        Args:
            answers (list): List of answers generated for each question.
            results (list): List of [question, true_label, generated_answer] for CSV output.
        """
        # Save to JSON
        with open(self.json_output_path, 'w', encoding='utf-8') as json_file:
            json.dump(answers, json_file, ensure_ascii=False, indent=4)

        # Save to CSV
        df = pd.DataFrame(results, columns=["Question", "True Answer", "Generated Answer"])
        df.to_csv(self.csv_output_path, index=False, encoding='utf-8')

In [ ]:
# CDM Task Implementation
class CommonDomainModelTask(Task):
    def create_input_template(self, text):
        """Format the input template for the CDM task."""
        return f"Provide a concise answer to the following question related to the Fintech Open Source Foundation's (FINOS) Common Domain Model (CDM): {text}"

    def run(self):
        """Perform the Common Domain Model (CDM) task."""
        system_prompt = "Deliver precise responses to questions about the Fintech Open Source Foundation's (FINOS) Common Domain Model (CDM)."

        answers, results = [], []
        data = pd.read_csv(self.file_path)
        x_test = data[self.col_question].astype(str).str.strip()
        y_true = data[self.col_label].astype(str).str.strip()

        for i in tqdm(range(len(x_test)), desc="CommonDomainModelTask"):
            question = x_test[i]
            user_prompt = self.create_input_template(question)

            try:
                response = self.inference.generate_response(system_prompt, user_prompt)
                answer = response
            except Exception as e:
                answer = f"Error: {str(e)}"

            answers.append(answer)
            results.append([x_test[i], y_true[i], answer])

        self.save_results(answers, results)

# MOF Task Implementation
class MOFTask(Task):
    def create_input_template(self, text, task_type="general"):
        """Format the input template for the MOF task."""
        if task_type == "abbreviation":
            return f"Expand the following MOF-related abbreviation into its full form: {text}."
        elif task_type == "approve":
            return f"Is the following license OSI-approved: {text}?"
        else:
            return f"Provide a concise answer to the following question about MOF's licensing requirements: {text}"

    def mof_task_abbreviation(self, question):
        """Handle MOF abbreviation expansion tasks."""
        system_prompt = "Act as the Model Openness Framework (MOF) expert, specializing in abbreviation expansion. Expand the following abbreviation into its most accurate full form within the MOF context. Provide only the full form, no explanations."
        return self.inference.generate_response(system_prompt, question)

    def mof_task_approve(self, question):
        """Handle MOF OSI approval status tasks."""
        system_prompt = "Act as the Model Openness Framework (MOF) expert, providing answers to questions about whether the license is OSI-approved or not. Provide only yes or no in the answer."
        return self.inference.generate_response(system_prompt, question)

    def mof_task_general(self, question):
        """Handle general MOF-related tasks."""
        system_prompt = "Act as the Model Openness Framework (MOF) expert, providing precise answers to questions about license requirements under the MOF and the content of the license. Respond accurately and concisely within the MOF context."
        return self.inference.generate_response(system_prompt, question)

    def run(self, task_type="general"):
        """Run the MOF task based on specified subtask type."""
        answers, results = [], []
        data = pd.read_csv(self.file_path)
        x_test = data[self.col_question].astype(str).str.strip()
        y_true = data[self.col_label].astype(str).str.strip()

        for i in tqdm(range(len(x_test)), desc="MOFTask"):
            question = x_test[i].replace("\n", "").strip()
            user_prompt = self.create_input_template(question, task_type)

            try:
                if task_type == "approve":
                    answer = self.mof_task_approve(user_prompt)
                elif task_type == "abbreviation":
                    answer = self.mof_task_abbreviation(user_prompt)
                else:
                    answer = self.mof_task_general(user_prompt)
            except Exception as e:
                answer = f"Error: {str(e)}"

            answers.append(answer)
            results.append([x_test[i], y_true[i], answer])

        self.save_results(answers, results)

# XBRL Task Implementation
class XBRLTask(Task):
    def create_input_template(self, text):
        """Format the input template for the XBRL task."""
        return f"Provide the exact answer to the following question: {text}?"

    def xbrl_task_term(self, question):
        """Handle XBRL terminology definition tasks."""
        system_prompt = "Provide a precise and concise definition of the term related to financial data extraction and application through XBRL (eXtensible Business Reporting Language) filings. Summarize its purpose, benefits, and primary uses in standardizing and sharing financial information within a single paragraph."
        return self.inference.generate_response(system_prompt, question)

    def xbrl_task_finmath(self, core_question, formula_name, formula):
        """Handle financial math calculations within XBRL data."""
        # Step 1: Generate calculation explanation
        system_prompt_cal = "Provide precise answers to detailed questions about financial data extraction and application using XBRL (eXtensible Business Reporting Language) filings, a standardized digital format for sharing and analyzing financial information. This task is to perform financial math. Ensure responses strictly match the correct answer without additional explanation. When answering questions about XBRL, it's essential to follow a structured approach. Here's how to methodically address these types of questions:"
        
        user_prompt_cal = (
            f"Question: {core_question}\n"
            f"Formula Name: {formula_name}\n"
            f"Formula: {formula}\n"
        )
        
        calculation = self.inference.generate_response(system_prompt_cal, user_prompt_cal)
        
        # Step 2: Generate final answer
        system_prompt_ans = "You are a financial expert tasked with carefully reading, analyzing, and answering the following eXtensible Business Reporting Language. Please follow the steps below:"
        
        user_prompt_ans = (
            f"Your task is to read the eXtensible Business Reporting Language (XBRL) question: {core_question} and find "
            f"the final answer based on the explanation provided: {calculation}. Provide only the final answer which is "
            f"the numerical result of the calculation. For formulas like ROI, provide percentages. Never use the percent symbol in percentages."
        )
        
        final_answer = self.inference.generate_response(system_prompt_ans, user_prompt_ans)
        return final_answer

    def xbrl_task_other(self, question):
        """Handle domain-specific, numeric, and tag queries."""
        system_prompt = "Provide precise answers to detailed questions about financial data extraction and application using XBRL (eXtensible Business Reporting Language) filings, a standardized digital format for sharing and analyzing financial information. This task covers three areas: domain-specific queries, numeric queries, and providing the correct US GAAP XBRL tags (e.g., 'US GAAP XBRL tag for revenue' should be answered as 'us-gaap:RevenueFromContractWithCustomerExcludingAssessedTax'). Ensure responses strictly match the correct answer without additional explanation."
        return self.inference.generate_response(system_prompt, question)

    def run(self, task_type="other"):
        """Run the XBRL task based on specified subtask type."""
        answers, results = [], []
        
        # Determine separator based on file extension
        _, file_extension = os.path.splitext(self.file_path)
        sep = '\t' if file_extension.lower() == '.tsv' else ','
        
        # Read data
        data = pd.read_csv(self.file_path, sep=sep)
        x_test = data[self.col_question].astype(str).str.strip()
        y_true = data[self.col_label].astype(str).str.strip()
        
        # Get formula columns for finmath tasks
        if task_type == "finmath":
            required_columns = ["Formula Name", "Formula"]
            for col in required_columns:
                if col not in data.columns:
                    raise ValueError(f"The required column '{col}' is missing in the input file.")
                    
            formula_name_col = data["Formula Name"].astype(str).str.strip()
            formula_col = data["Formula"].astype(str).str.strip()
        else:
            formula_name_col = None
            formula_col = None
            
        for i in tqdm(range(len(x_test)), desc="XBRLTask"):
            question = str(x_test[i]).replace("\n", "").strip()
            
            try:
                if task_type == "finmath":
                    formula_name = str(formula_name_col[i]).strip()
                    formula = str(formula_col[i]).strip()
                    user_prompt = self.create_input_template(question)
                    answer = self.xbrl_task_finmath(question, formula_name, formula)
                elif task_type == "term":
                    user_prompt = self.create_input_template(question)
                    answer = self.xbrl_task_term(user_prompt)
                else:  # 'other' for domain, numeric, tag queries
                    user_prompt = self.create_input_template(question)
                    answer = self.xbrl_task_other(user_prompt)
            except Exception as e:
                answer = f"Error: {str(e)}"
                
            answers.append(answer)
            results.append([x_test[i], y_true[i], answer])
            
        self.save_results(answers, results)

In [ ]:
# Main Function
def main():
    # Create output directories
    save_val_dir = "inference_results"
    save_path_csv = os.path.join(save_val_dir, "csv")
    save_path_json = os.path.join(save_val_dir, "json")
    os.makedirs(save_path_csv, exist_ok=True)
    os.makedirs(save_path_json, exist_ok=True)

    # Define task configurations
    tasks = [
        # XBRL Tasks
        {
            "class": XBRLTask,
            "file_path": "test_data/Task7-XBRL Terminology-testing.csv",
            "col_question": "Term",
            "col_label": "Explanation",
            "csv_output_path": os.path.join(save_path_csv, "xbrl_task_term.csv"),
            "json_output_path": os.path.join(save_path_json, "xbrl_task_term.json"),
            "params": {"task_type": "term"}
        },
        {
            "class": XBRLTask,
            "file_path": "test_data/Task7-XBRL-domain_numberic_query-testing.csv",
            "col_question": "question",
            "col_label": "answer",
            "csv_output_path": os.path.join(save_path_csv, "xbrl_task_numeric_query.csv"),
            "json_output_path": os.path.join(save_path_json, "xbrl_task_numeric_query.json"),
            "params": {"task_type": "other"}
        },
        {
            "class": XBRLTask,
            "file_path": "test_data/Task7-XBRL-financial_formula_calc-XBRL-tags-testing.csv",
            "col_question": "query",
            "col_label": "answer",
            "csv_output_path": os.path.join(save_path_csv, "xbrl_task_formula_tag.csv"),
            "json_output_path": os.path.join(save_path_json, "xbrl_task_formula_tag.json"),
            "params": {"task_type": "tag_ratio"}
        },
        {
            "class": XBRLTask,
            "file_path": "test_data/Task7-XBRL-financial_math-testing_format_limited.tsv",
            "col_question": "Question",
            "col_label": "Answer",
            "csv_output_path": os.path.join(save_path_csv, "xbrl_task_finmath.csv"),
            "json_output_path": os.path.join(save_path_json, "xbrl_task_finmath.json"),
            "params": {"task_type": "finmath"}
        },
        {
            "class": XBRLTask,
            "file_path": "test_data/Task7-XBRL-Tag_Query_to_XBRL_Reports-testing.csv",
            "col_question": "query",
            "col_label": "answer",
            "csv_output_path": os.path.join(save_path_csv, "xbrl_task_tag_query.csv"),
            "json_output_path": os.path.join(save_path_json, "xbrl_task_tag_query.json"),
            "params": {"task_type": "other"}
        },
        # Common Domain Model Task
        {
            "class": CommonDomainModelTask,
            "file_path": "test_data/Task8-CDM-testing.csv",
            "col_question": "Question",
            "col_label": "Answer",
            "csv_output_path": os.path.join(save_path_csv, "cdm_task.csv"),
            "json_output_path": os.path.join(save_path_json, "cdm_task.json"),
            "params": {}
        },
        # MOF Tasks
        {
            "class": MOFTask,
            "file_path": "test_data/Task9_MOF-License_OSI_Approval-testing.csv",
            "col_question": "Question",
            "col_label": "answer",
            "csv_output_path": os.path.join(save_path_csv, "mof_task_approval.csv"),
            "json_output_path": os.path.join(save_path_json, "mof_task_approval.json"),
            "params": {"task_type": "approve"}
        },
        {
            "class": MOFTask,
            "file_path": "test_data/Task9-MOF-Detailed_QA-testing.csv",
            "col_question": "Question",
            "col_label": "answer",
            "csv_output_path": os.path.join(save_path_csv, "mof_task_detailed.csv"),
            "json_output_path": os.path.join(save_path_json, "mof_task_detailed.json"),
            "params": {"task_type": "general"}
        },
        {
            "class": MOFTask,
            "file_path": "test_data/Task9-MOF-License_Abbreviation-testing.csv",
            "col_question": "Question",
            "col_label": "answer",
            "csv_output_path": os.path.join(save_path_csv, "mof_task_abbreviation.csv"),
            "json_output_path": os.path.join(save_path_json, "mof_task_abbreviation.json"),
            "params": {"task_type": "abbreviation"}
        },
    ]

  # Loop through each task and run inference for all models
    for task_config in tasks:
        task_class = task_config["class"]
        for engine in inference_engines:
            model_name = engine.name
            print(f"\nRunning task '{task_class.__name__}' with model '{model_name}'")

            # Create output file paths with model name suffix
            csv_output_path = task_config["csv_output_path"].replace(".csv", f"_{model_name}.csv")
            json_output_path = task_config["json_output_path"].replace(".json", f"_{model_name}.json")

            # Initialize the task with the current inference engine and updated output paths
            task_instance = task_class(
                inference_engine=engine,
                file_path=task_config["file_path"],
                col_question=task_config["col_question"],
                col_label=task_config["col_label"],
                csv_output_path=csv_output_path,
                json_output_path=json_output_path
            )

            # Run the task with any additional parameters
            task_instance.run(**task_config.get("params", {}))

    print("\nAll tasks completed for all models.")



In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
!zip -r inference_results.zip inference_results/

In [ ]:
from google.colab import files
files.download("inference_results.zip")

In [ ]:
!rm -r inference_results/

In [ ]:
!rm -r test_data/